In [1]:
# load libraries
import pandas as pd
import geopandas as gpd
from shapely.geometry import shape
import numpy as np
# gpd .sjoin() function requires installing the
# Rtree python module with the libspatialindex library via the command: pip install rtree

### Postcodes
#### we use the NSPL to obtain postcodes for each nation, which has the new 2021 boundaries (except for Scotland)

In [2]:
# read NSPL postcodes with LSOAs, lat & long -- boundaries as of censuses in 2021
nspl21 = pd.read_csv("data/geos/NSPL21_AUG_2023_UK/Data/NSPL21_AUG_2023_UK.csv", low_memory = False)

In [5]:
nspl21.head()

,pcds,doterm,ctry,lsoa21,lat,long,geometry
2656,AB10 1AB,NaN,S92000003,S01006646,57.149606,-2.096916,POINT (-2.09692 57.14961)
2658,AB10 1AF,NaN,S92000003,S01006646,57.149590,-2.096923,POINT (-2.09692 57.14959)
2659,AB10 1AG,NaN,S92000003,S01006646,57.149051,-2.097004,POINT (-2.09700 57.14905)
2660,AB10 1AH,NaN,S92000003,S01006646,57.149590,-2.096923,POINT (-2.09692 57.14959)
2662,AB10 1AL,NaN,S92000003,S01006646,57.150058,-2.095916,POINT (-2.09592 57.15006)


In [4]:
# data processing
# select only relevant columns and filter for England, Wales, N Ireland and Scotland "ctry" codes
nspl21 = nspl21[["pcds", "doterm", "ctry", "lsoa21", "lat", "long"]]

ctry_codes = ["E92000001","W92000004","N92000002","S92000003"]
nspl21 = nspl21[nspl21["ctry"].isin(ctry_codes)]

# "doterm" not null means postcode is terminated - filter to keep NaNs (active postcodes)
nspl21 = nspl21[nspl21["doterm"].isna()]

# convert to geodataframe using 'lat' and 'long' (X is longitude, Y is latitude) - no CRS
nspl21_gdf = gpd.GeoDataFrame(nspl21, geometry = gpd.points_from_xy(nspl21['long'], nspl21['lat']))

# filter to create England, Wales, and N Ireland postcode tables
pcd_gdf_E = nspl21_gdf[nspl21_gdf["ctry"] == "E92000001"] # England
pcd_gdf_W = nspl21_gdf[nspl21_gdf["ctry"] == "W92000004"] # Wales
pcd_gdf_NI = nspl21_gdf[nspl21_gdf["ctry"] == "N92000002"] # N Ireland

In [ ]:
# save full nspl as shapefile if wanted - takes a while
# nspl21_gdf.to_file(filename = 'NSPL_postcodes_shp/nspl_postcodes.shp', driver = 'ESRI Shapefile')

In [7]:
print(nspl21_gdf.crs)

None


In [8]:
pcd_gdf_E.head()

,pcds,doterm,ctry,lsoa21,lat,long,geometry
39163,AL1 1AG,NaN,E92000001,E01023741,51.745290,-0.328628,POINT (-0.32863 51.74529)
39165,AL1 1AJ,NaN,E92000001,E01023741,51.744498,-0.328599,POINT (-0.32860 51.74450)
39170,AL1 1AR,NaN,E92000001,E01023684,51.739727,-0.317492,POINT (-0.31749 51.73973)
39171,AL1 1AS,NaN,E92000001,E01023726,51.749073,-0.335471,POINT (-0.33547 51.74907)
39172,AL1 1AT,NaN,E92000001,E01023682,51.742011,-0.319421,POINT (-0.31942 51.74201)


### Old "LSOA" boundaries

In [9]:
# read all UK old boundaries shapefile
all_old = gpd.read_file("data/geos/infuse_lsoa_lyr_2011/infuse_lsoa_lyr_2011.shp")

In [11]:
all_old.head()

,geo_code,geo_label,geo_labelw,label,name,geometry
0,E01003513,Newham 035D,NaN,E92000001E09000025E01003513,Newham 035D,"POLYGON ((541893.189 181249.621, 541900.568 18..."
1,E01031647,Horsham 002D,NaN,E92000001E07000227E01031647,Horsham 002D,"POLYGON ((518376.682 132574.695, 518375.785 13..."
2,E01022006,Tendring 002C,NaN,E92000001E07000076E01022006,Tendring 002C,"POLYGON ((623754.716 231042.037, 623759.750 23..."
3,E01001159,Croydon 002C,NaN,E92000001E09000008E01001159,Croydon 002C,"POLYGON ((532233.977 170474.976, 532229.824 17..."
4,E01008088,Sheffield 012B,NaN,E92000001E08000019E01008088,Sheffield 012B,"POLYGON ((433539.233 392096.845, 433539.125 39..."


In [12]:
# data processing
# select only relevant columns
all_old = all_old[["geo_code", "geometry"]]

# create new "country" column based on LSOA codes using for loop
# new empty 'country' column
all_old['country'] = ""
# dictionary for code to country
code_to_country = {
    'E': 'England',
    'W': 'Wales',
    'S': 'Scotland'}

# Iterate through rows and fill the 'country' column based on the first letter of 'geo_code'
for i, row in all_old.iterrows():
    #print(i, row)
    first_letter = row['geo_code'][0] # get first letter of geo_code
    country = code_to_country.get(first_letter, 'Northern Ireland')  # Default to 'Northern Ireland if not found
    all_old.at[i, 'country'] = country

# filter to create old England, Wales, and N Ireland boundaries tables
old_E = all_old[all_old["country"] == "England"] # England
old_W = all_old[all_old["country"] == "Wales"] # Wales
old_NI = all_old[all_old["country"] == "Northern Ireland"] # N Ireland

In [13]:
print(all_old.crs)

epsg:27700


In [14]:
old_NI.head()

,geo_code,geometry,country
14,95YY06W1,"POLYGON ((45250.799 524289.846, 45249.238 5242...",Northern Ireland
91,95QQ10W1,"POLYGON ((146997.062 570813.338, 146990.814 57...",Northern Ireland
98,95WW21S2,"POLYGON ((148311.545 539035.713, 148329.700 53...",Northern Ireland
145,95SS24S1,"POLYGON ((126372.098 517640.937, 126362.575 51...",Northern Ireland
178,95GG25S3,"POLYGON ((141174.600 527586.497, 141165.305 52...",Northern Ireland


### New "LSOA" boundaries

In [15]:
# read England and Wales new LSOA boundaries shapefile
new = gpd.read_file("data/geos/Eng_Wales_LSOA_2021_Boundaries/LSOA_2021_EW_BGC.shp")

In [18]:
new.head()

,LSOA21CD,geometry,country
0,E01000001,"POLYGON ((532105.312 182010.574, 532162.491 18...",England
1,E01000002,"POLYGON ((532634.497 181926.016, 532619.141 18...",England
2,E01000003,"POLYGON ((532135.138 182198.131, 532158.250 18...",England
3,E01000005,"POLYGON ((533808.018 180767.774, 533649.037 18...",England
4,E01000006,"POLYGON ((545122.049 184314.931, 545271.849 18...",England


In [17]:
# data processing
# select only relevant columns
new = new[["LSOA21CD", "geometry"]]

# create new "country" column based on LSOA codes using for loop
# new empty 'country' column
new['country'] = ""

# Iterate through rows and fill the 'country' column based on the first letter of 'LSOA21CD'
for i, row in new.iterrows():
    #print(i, row)
    first_letter = row['LSOA21CD'][0] # get first letter of "LSOA21CD"
    country = code_to_country.get(first_letter)  # using same code_to_country dictionary from before
    new.at[i, 'country'] = country
    
# filter to create new England and Wales boundaries tables
new_E = new[new["country"] == "England"] # England
new_W = new[new["country"] == "Wales"] # Wales

In [19]:
print(new.crs)

epsg:27700


In [20]:
new_W.head()

,LSOA21CD,geometry,country
33755,W01000003,"POLYGON ((244811.204 393835.536, 244811.100 39...",Wales
33756,W01000004,"POLYGON ((241027.292 394771.247, 241064.790 39...",Wales
33757,W01000005,"POLYGON ((259509.403 379193.781, 259564.297 37...",Wales
33758,W01000006,"POLYGON ((241039.000 381748.755, 241326.500 38...",Wales
33759,W01000007,"MULTIPOLYGON (((242547.168 368906.395, 242165....",Wales


In [21]:
# read Northern Ireland new SDZ boundaries shapefile
new_NI = gpd.read_file("data/geos/NI-sdz2021-esri-shapefile/SDZ2021.shp")

In [24]:
new_NI.head()

,SDZ2021_cd,geometry
0,N21000001,"POLYGON ((302828.621 393821.676, 302831.930 39..."
1,N21000002,"POLYGON ((319895.281 396949.987, 319893.771 39..."
2,N21000003,"POLYGON ((302824.752 393819.021, 302828.603 39..."
3,N21000004,"POLYGON ((307690.541 389508.860, 307599.464 38..."
4,N21000005,"POLYGON ((308921.837 390709.973, 308934.877 39..."


In [23]:
# select only relevant columns
new_NI = new_NI[["SDZ2021_cd", "geometry"]]

In [25]:
print(new_NI.crs)

epsg:29902


In [26]:
len(new_NI)

850

### Function

In [53]:
new_E.head()

,LSOA21CD,geometry,country
0,E01000001,"POLYGON ((532105.312 182010.574, 532162.491 18...",England
1,E01000002,"POLYGON ((532634.497 181926.016, 532619.141 18...",England
2,E01000003,"POLYGON ((532135.138 182198.131, 532158.250 18...",England
3,E01000005,"POLYGON ((533808.018 180767.774, 533649.037 18...",England
4,E01000006,"POLYGON ((545122.049 184314.931, 545271.849 18...",England


In [54]:
# global variables -- need to be defined each time

old_code_col = "geo_code" # column name for old boundaries code
new_code_col = "LSOA21CD" # column name for new boundaries code
pcd_col = "pcds" # column name for postcodes
pcd_new_lsoa = "lsoa21" # column name for new boundaries code in postcodes table

In [35]:
# function to translate old boundaries to new boundaries using proportion of postcodes
# the function requires 3 inputs: old boundaries, new boundaries and postcodes, all as .shp
# all the required column names also need to be defined outside the function (global environment)

def translate_geos(old_boundaries, new_boundaries, postcodes):
    # re-project all geometries to matching CRS
    postcodes = postcodes.set_crs(4326)
    old_boundaries = old_boundaries.to_crs(4326)
    new_boundaries = new_boundaries.to_crs(4326)
    
    # get number of postcodes in new boundaries
    new_postcode_count = postcodes.groupby([pcd_new_lsoa]).count()[[pcd_col]].reset_index()
    new_postcode_count = new_postcode_count.rename(columns={pcd_col:"sum_new_postcodes"}) # rename postcode count col
    #print(new_postcode_count)
    
    # get intersection of old boundaries and new boundaries
    # returns the intersection polygons for each old boundary part within a new boundary
    new_w_old = gpd.overlay(new_boundaries, old_boundaries, how = 'intersection',
                            keep_geom_type = False)
    #print(new_w_old)
    
    # left-join postcodes found in each intersection area
    new_w_old_postcodes = gpd.sjoin(new_w_old, postcodes[[pcd_col, "geometry"]].to_crs(new_w_old.crs),
                                    how = "left")                 # .to_crs() so postcodes CRS matches
    #print(new_w_old_postcodes)
    
    # count number of postcodes in each intersection area 
    new_w_old_postcodes = new_w_old_postcodes.groupby([new_code_col, old_code_col]).count()[[pcd_col]].reset_index()
    new_w_old_postcodes = new_w_old_postcodes.rename(columns={pcd_col: "old_in_new_postcodes"}) # rename postcodes col
    #print(new_w_old_postcodes)
    
    # create new table:
    # proportion of postcodes in each old boundary part as proportion of total postcodes in new boundary
    proportions = pd.merge(new_postcode_count, new_w_old_postcodes, how = "left",
                          left_on = pcd_new_lsoa, right_on = new_code_col)
    proportions['prop'] = proportions["old_in_new_postcodes"]/proportions["sum_new_postcodes"]

    return proportions   

In [33]:
translate_geos(old_W, new_W, pcd_gdf_W)

         lsoa21  sum_new_postcodes
0     W01000003                 67
1     W01000004                 61
2     W01000005                 96
3     W01000006                 66
4     W01000007                115
...         ...                ...
1912  W01002036                 30
1913  W01002037                 22
1914  W01002038                 32
1915  W01002039                 22
1916  W01002040                 39

[1917 rows x 2 columns]
        LSOA21CD country_1   geo_code country_2  \
0      W01000003     Wales  W01000003     Wales   
1      W01000004     Wales  W01000003     Wales   
2      W01000021     Wales  W01000003     Wales   
3      W01000003     Wales  W01000004     Wales   
4      W01000004     Wales  W01000004     Wales   
...          ...       ...        ...       ...   
12285  W01001995     Wales  W01001958     Wales   
12286  W01001996     Wales  W01001958     Wales   
12287  W01001997     Wales  W01001958     Wales   
12288  W01002002     Wales  W01000544     Wal

,lsoa21,sum_new_postcodes,LSOA21CD,geo_code,old_in_new_postcodes,prop
0,W01000003,67,W01000003,W01000003,67,1.000000
1,W01000003,67,W01000003,W01000004,0,0.000000
2,W01000003,67,W01000003,W01000021,0,0.000000
3,W01000004,61,W01000004,W01000003,0,0.000000
4,W01000004,61,W01000004,W01000004,61,1.000000
...,...,...,...,...,...,...
12285,W01002040,39,W01002040,W01001061,36,0.923077
12286,W01002040,39,W01002040,W01001062,0,0.000000
12287,W01002040,39,W01002040,W01001066,0,0.000000
12288,W01002040,39,W01002040,W01001073,0,0.000000


In [55]:
# get England translate_geos
# remember to change global variables above
translate_england = translate_geos(old_E, new_E, pcd_gdf_E)

In [56]:
translate_england

,lsoa21,sum_new_postcodes,LSOA21CD,geo_code,old_in_new_postcodes,prop
0,E01000001,57,E01000001,E01000001,57,1.000000
1,E01000001,57,E01000001,E01000002,0,0.000000
2,E01000001,57,E01000001,E01000003,0,0.000000
3,E01000001,57,E01000001,E01032739,0,0.000000
4,E01000001,57,E01000001,E01032740,2,0.035088
...,...,...,...,...,...,...
223570,E01035762,78,E01035762,E01028507,0,0.000000
223571,E01035762,78,E01035762,E01028806,0,0.000000
223572,E01035762,78,E01035762,E01028824,0,0.000000
223573,E01035762,78,E01035762,E01028825,0,0.000000


In [58]:
translate_england[["lsoa21", "prop"]].groupby(["lsoa21"]).sum()

,prop
lsoa21,
E01000001,1.035088
E01000002,0.955556
E01000003,1.000000
E01000005,0.990099
E01000006,1.000000
...,...
E01035758,0.977778
E01035759,0.983871
E01035760,1.000000


In [59]:
# write to .csv
translate_england.to_csv("translate_england.csv")

In [36]:
# get Wales translate_geos
# remember to change global variables above
translate_wales = translate_geos(old_W, new_W, pcd_gdf_W)

In [37]:
translate_wales

,lsoa21,sum_new_postcodes,LSOA21CD,geo_code,old_in_new_postcodes,prop
0,W01000003,67,W01000003,W01000003,67,1.000000
1,W01000003,67,W01000003,W01000004,0,0.000000
2,W01000003,67,W01000003,W01000021,0,0.000000
3,W01000004,61,W01000004,W01000003,0,0.000000
4,W01000004,61,W01000004,W01000004,61,1.000000
...,...,...,...,...,...,...
12285,W01002040,39,W01002040,W01001061,36,0.923077
12286,W01002040,39,W01002040,W01001062,0,0.000000
12287,W01002040,39,W01002040,W01001066,0,0.000000
12288,W01002040,39,W01002040,W01001073,0,0.000000


In [43]:
translate_wales[["lsoa21", "prop"]].groupby(["lsoa21"]).sum()

,prop
lsoa21,
W01000003,1.000000
W01000004,1.000000
W01000005,1.000000
W01000006,1.000000
W01000007,1.000000
...,...
W01002036,0.933333
W01002037,1.045455
W01002038,1.031250


In [44]:
# write to .csv
translate_wales.to_csv("translate_wales.csv")

In [48]:
# get Northern Ireland translate_geos
# remember to change global variables above
translate_n_ire = translate_geos(old_NI, new_NI, pcd_gdf_NI)

In [49]:
translate_n_ire

,lsoa21,sum_new_postcodes,SDZ2021_cd,geo_code,old_in_new_postcodes,prop
0,N21000001,89,N21000001,95AA05W1,1,0.011236
1,N21000001,89,N21000001,95AA07W1,32,0.359551
2,N21000001,89,N21000001,95AA12W1,7,0.078652
3,N21000001,89,N21000001,95AA13S1,0,0.000000
4,N21000001,89,N21000001,95AA13S2,1,0.011236
...,...,...,...,...,...,...
4631,N21000849,59,N21000849,95BB20S1,24,0.406780
4632,N21000849,59,N21000849,95BB20S2,0,0.000000
4633,N21000849,59,N21000849,95BB21S2,3,0.050847
4634,N21000850,62,N21000850,95BB20S1,32,0.516129


In [50]:
translate_n_ire[["lsoa21", "prop"]].groupby(["lsoa21"]).sum()

,prop
lsoa21,
N21000001,1.0
N21000002,1.0
N21000003,1.0
N21000004,1.0
N21000005,1.0
...,...
N21000846,1.0
N21000847,1.0
N21000848,1.0


In [51]:
# write to .csv
translate_n_ire.to_csv("translate_n_ireland.csv")